In [ ]:
%pylab inline
import matplotlib.pyplot as plt

#import matplotlib
#matplotlib.style.use('ggplot') ## gnuplot style

import numpy as np
import pandas as pd

import warnings as wn
import partial_corr # found at 
                    # https://gist.github.com/fabianp/9396204419c7b638d38f


#size
width, height=12,8
plt.rcParams['figure.figsize'] = width, height #aggiunta pt

#precision in tables
pd.set_option('precision',2)
#rows in tables
pd.set_option('display.max_rows', 1000)

# this is used for regression below; install (via pip) statsmodels and patsy
import statsmodels.formula.api as sm

In [ ]:
# to chose a csv file in the current folder
import os
filesHere=os.listdir("./")
selected=[]
for i in range(len(filesHere)):
    if filesHere[i].find('_ts.csv')>0: selected.append(filesHere[i])
selected.sort()
for i in range(len(selected)):
    print (i, selected[i])

In [ ]:
num=int(input("Choose a file via its number (>=0;<="+str(len(selected)-1)+" "))

try:
    modPars_df = pd.read_csv(selected[num][:17]+'_modPars.csv')
    modPars_df.index += 1 
except BaseException:
    modPars_df = pd.DataFrame([["no changes in parameters"]],columns=["  "])
    modPars_df.index += 1

firms=False
    
try:
    firms_df = pd.read_csv(selected[num][:17]+'_firms.csv')
    modPars_df.index += 1 
    firms=True
except BaseException:
    pass


par_df = pd.read_csv(selected[num][:17]+'_par.csv')
par_df.index += 1 

ts_df = pd.read_csv(selected[num])
#set index to start from 1, data are collected at the end of each period
ts_df.index += 1 

str_df = pd.read_csv(selected[num][:17]+'_str.csv')
#leave index to start from 0, data are collected at the beginning of each period

***Parameters***

In [ ]:
par_df.astype(str,errors='ignore')

***Modified parameters***

In [ ]:
modPars_df.astype(str,errors='ignore')

***Time series, data collected at the end of each period***

In [ ]:
ts_df

In [ ]:
ts_df.describe()

In [ ]:
ts_df.corr(method="pearson").style.format("{:.2}")

The origin of the partial_corr source is [https://gist.github.com/fabianp/9396204419c7b638d38f](https://gist.github.com/fabianp/9396204419c7b638d38f)

At [http://en.wikipedia.org/wiki/Partial_correlation#Using_linear_regression](http://en.wikipedia.org/wiki/Partial_correlation#Using_linear_regression) we have the explanation of the need of augmenting the data matrix with a 1 to allow for a constant term in the regression.

In [ ]:
wn.filterwarnings(action="ignore") # to eliminate a warning about 
                                   #LAPACK lib

np.set_printoptions(precision=3,suppress=True)
ts=ts_df.values
ts_int = np.hstack((np.ones((ts.shape[0],1)), ts))

partial_corr.partial_corr(ts_int)[1:, 1:]

In [ ]:
ts=ts_df.drop(columns="plannedProduction").values
ts_int = np.hstack((np.ones((ts.shape[0],1)), ts))

partial_corr.partial_corr(ts_int)[1:, 1:]

In [ ]:
ts=ts_df.drop(columns="totalProduction").values
ts_int = np.hstack((np.ones((ts.shape[0],1)), ts))

partial_corr.partial_corr(ts_int)[1:, 1:]

In [ ]:
result = sm.ols(formula="totalProfit ~ price + wage + totalProduction + unemployed", \
                data=ts_df).fit()

print (result.summary())

***Structural infos, data collected at the beginning of each period***

In [ ]:
str_df

In [ ]:
myPlot = ts_df.plot(figsize=(11,8),secondary_y=['hPriceSd', 'price','wage'],marker="*",
color=["OrangeRed","LawnGreen","Blue","Violet","lightblue","Pink","Gray","Brown"])
myPlot.set_ylabel('unemployed, totalProfit, totalProduction, plannedProduction, consumptionQ')
myPlot.right_ax.set_ylabel('hPriceSd, price, wage')
myPlot.legend(loc='upper left') #, bbox_to_anchor=(-0.35, 0.5)
myPlot.axes.right_ax.legend(loc='lower right') #, bbox_to_anchor=(1.1, 0.5)

In [ ]:
str_df.plot(figsize=(11,8),secondary_y='workers',marker="*",color=["r","b"])

Best solutions to produce a LaTeX table from these data (the example is related to ts_df.corr table):

    corr=ts_df.corr(method='pearson')
    print corr.to_latex()
    
    "print" to have the output nicely formatted; copy and paste it to LaTeX and the 
    result works.
    
To use caption and label with tabular simply include that output within:

    \begin{table}
    
    output above
    
    \label{a label}
    \caption{a caption}
    \end{table}

    You can add the usual [htbp] specification as \begin{table}[htbp]


Other solutions:
1. online [http://www.tablesgenerator.com](http://www.tablesgenerator.com), reading the csv file;
2. using a converter as [http://html2latex.sourceforge.net](http://html2latex.sourceforge.net).

## The first method is applied in the cell below.

In [ ]:
corr=ts_df.corr(method='pearson')
def ff(x):
    return '%1.2f' % x
print (corr.to_latex(formatters=[ff,ff,ff,ff,ff,ff,ff,ff]))

## *Data from each firm in each period*

In [ ]:
if firms: print(firms_df.describe())
else: print('no data for each firm in each period')